# Advanced Search Techniques with Azure AI Search: Keyword, Vector, and Hybrid Methods

This notebook demonstrates how to perform different types of searches using Azure AI Search, including keyword search, vector search, hybrid search, semantic ranking, and query rewriting.

## Prerequisites

Before running the notebook, ensure you have the following: 

- [Clone](https://github.com/abbhakan/rag-time) the repository to your local machine by following the script below:

    ```bash
    git clone git@github.com:abbhakan/rag-time.git
    cd rag-time
    ```

- Journey 2!!
- Endre .env.sample til -> .env og plugg inn admin_key fra teams
- Install Python 3.8 or later from [python.org](https://python.org).

## Steps to Use the Notebook

### 1. Install Required Libraries

Run the first code cell to install the required Python libraries:

In [ ]:
#windows
%pip install azure-search-documents==11.6.0b9 azure-identity python-dotenv pandas jinja2 --quiet

In [ ]:
#mac
!python3 -m pip install azure-search-documents==11.6.0b9 azure-identity python-dotenv pandas jinja2 --quiet

### 2. Set Up Environment Variables

To store credentials securely, rename `.env.sample` file to `.env` in the same directory as the notebook and update the following variables:

In [ ]:
AZURE_SEARCH_SERVICE_ENDPOINT="https://aisearch-fastai-rag.search.windows.net"
AZURE_SEARCH_INDEX="sopravector2"
AZURE_SEARCH_ADMIN_KEY=""

After setting up, the notebook will automatically load these values using dotenv.

### 3. Load Environment Variables

Run the following command to load environment variables from the .env file:

In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY"))

This will ensure all necessary credentials are available before setting up the API client.

### 4. Set Up API Client and Define the Display Function

Initialize the Azure AI Search Client for interacting with the Azure Search service and make the search results easier to read by defining a function that formats and displays results:

In [2]:
from azure.search.documents import SearchClient
import pandas as pd

search_client = SearchClient(endpoint, index_name, credential)

def display_results(results):
    df = pd.json_normalize(list(results)).dropna(axis=1, how='all')
    df["chunk"] = df["chunk"].apply(lambda c: c[:300] + '...' if len(c) > 300 else c)
    first_cols = ['title', 'chunk', '@search.score']
    df = df[first_cols + [col for col in df.columns if col not in first_cols]]

    df = df.style.set_properties(**{
        'max-width': '500px',
        'text-align': 'left',
        'white-space': 'normal',
        'word-wrap': 'break-word'
    }).hide(axis="index")


    return df

##### Søke-felt #####
searchText = "Pappapermisjon i sopra steria"
#####################

/Users/johan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### 5. Perform Different Search Methods

#### Keyword Search

Execute a traditional keyword-based search:

In [3]:
results = search_client.search(search_text=searchText, top=5, select=["title", "chunk"])

display_results(results)


title,chunk,@search.score
1. Hvilke retningslinjer gjelder hos oss_.pdf,"underlagt et familiemedlem. Tidligere ansatte Sopra Steria har som policy å unngå at tidligere ansatte, som etter avsluttet ansettelse i Sopra Steria har startet sin egen 4/23/25, 9:09 AM 1. Hvilke retningslinjer gjelder hos oss? https://steria.sharepoint.com/sites/Personalhndbok/SitePages/1. Hv...",3.698846
20. Hva skjer når jeg skal slutte_.pdf,"å fysisk møte opp på Sopra Steria sin lokasjon for å levere inn alle utleverte effekter, dokumenter etc. som tilhører Sopra Steria. Du plikter også å gjøre opp eventuelle tilgodehavender som Sopra Steria måtte ha. Det er vanlig at en representant for HR gjennomfører en sluttsamtale. Hva skjer når ...",3.489025
1. Hvilke retningslinjer gjelder hos oss_.pdf,som ikke går ut over virksomhetens omdømme. Ansatte er å betrakte som representanter for virksomheten i slike sammenhenger. 4. I forbindelse med representasjon følger vi Statens regulativ. 5. Alkoholservering i Sopra Steria sine lokaler i arbeidstiden mellom kl. 08-16 er ikke tillatt. 4/23/25...,3.377539
"18. Helse, miljø og sikkerhet.pdf","18. Helse, miljø og sikkerhet  Arbeidsmiljø  Verneombud  Arbeidsmiljøutvalget Arbeidsmiljø I Sopra Steria søker vi å ha et arbeidsmiljø preget av likeverd, respekt og åpenhet. Vi godtar ikke diskriminering på bakgrunn av kjønn, nasjonalitet, etnisitet, religion, alder, legning, sivil status el...",3.052828
9. Ferie.pdf,"Alle ansatte skal avvikle sin hovedferie, minimum fire uker, i juli og ellers i stille perioder. Dette gjelder også medarbeidere som har opparbeidet feriepenger hos tidligere arbeidsgiver. Ferie utover hovedferien skal legges til jul, påske og ellers stille perioder i prosjektet, eller til periode...",3.019975


#### Vector Search

Retrieve documents using vector similarity search:

In [4]:
from azure.search.documents.models import VectorizableTextQuery

results = search_client.search(
    None,
    top=5,
    vector_queries=[VectorizableTextQuery(text=searchText, k_nearest_neighbors=50, fields="text_vector")],
    select=["title", "chunk"]
)

display_results(results)

title,chunk,@search.score
14. Foreldrepermisjon.pdf,"i permisjon. Goder under foreldrepermisjon Du beholder dine goder som forsikringer, mobilabonnement og pensjon som vanlig mens du er i foreldrepermisjon. Sopra Steria dager tjenes ikke opp mens du er i permisjon. 4/23/25, 9:11 AM 14. Foreldrepermisjon https://steria.sharepoint.com/sites/Persona...",0.722310
15. Andre permisjoner.pdf,inn i Workday og sendes til godkjenning hos leder. Nærmeste leder kan etter søknad innvilge inntil tre dager permisjon uten lønn. Ved godkjenning sendes forespørselen til Worktime og genererer din timeføring. Timene går til trekk i lønn inneværende eller påfølgende måned. Lengre permisjoner uten...,0.708640
14. Foreldrepermisjon.pdf,"Velferdspermisjon i forbindelse med fødsel Far eller medmor får to uker permisjon med lønn i forbindelse med fødsel. Du informerer din leder, og registrerer dette på 2 weeks in relation to child birth i Workday. Du trenger ikke informere HR om disse to ukene. Merk at permisjonen er to kalenderuker,...",0.700860
14. Foreldrepermisjon.pdf,14. Foreldrepermisjon  Hva må du gjøre når du skal ut i…  Endring i foreldrepermisjon  Arbeid i foreldrepermisjon  Syk i foreldrepermisjon  Velferdpermisjon fødsel  Adopsjon  Hva gjelder i permisjonstiden?  Ferie i permisjonstiden  Fri ved amming Hva må du gjøre når du skal ut i foreld...,0.688183
15. Andre permisjoner.pdf,"For vernepliktige og utskrevet befal skal det, når tjenesten varer mer enn 1 uke (7 dager), foretas et lønnstrekk som svarer til den militære grad/stillings hovedregulativlønn. Lønnstrekket kan ikke overstige vanlig lønn for perioden. Kopi av lønnsslipp sendes til payroll. Du vil få informasjon i bj...",0.683208


#### Hybrid Search (Keyword + Vector Search)

Combine keyword and vector searches for better accuracy:

In [5]:
results = search_client.search(
    search_text=searchText,
    vector_queries=[VectorizableTextQuery(text=searchText, k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"]
)

display_results(results)

title,chunk,@search.score
20. Hva skjer når jeg skal slutte_.pdf,"å fysisk møte opp på Sopra Steria sin lokasjon for å levere inn alle utleverte effekter, dokumenter etc. som tilhører Sopra Steria. Du plikter også å gjøre opp eventuelle tilgodehavender som Sopra Steria måtte ha. Det er vanlig at en representant for HR gjennomfører en sluttsamtale. Hva skjer når ...",0.031545
15. Andre permisjoner.pdf,inn i Workday og sendes til godkjenning hos leder. Nærmeste leder kan etter søknad innvilge inntil tre dager permisjon uten lønn. Ved godkjenning sendes forespørselen til Worktime og genererer din timeføring. Timene går til trekk i lønn inneværende eller påfølgende måned. Lengre permisjoner uten...,0.031319
14. Foreldrepermisjon.pdf,"Velferdspermisjon i forbindelse med fødsel Far eller medmor får to uker permisjon med lønn i forbindelse med fødsel. Du informerer din leder, og registrerer dette på 2 weeks in relation to child birth i Workday. Du trenger ikke informere HR om disse to ukene. Merk at permisjonen er to kalenderuker,...",0.030018
1. Hvilke retningslinjer gjelder hos oss_.pdf,"underlagt et familiemedlem. Tidligere ansatte Sopra Steria har som policy å unngå at tidligere ansatte, som etter avsluttet ansettelse i Sopra Steria har startet sin egen 4/23/25, 9:09 AM 1. Hvilke retningslinjer gjelder hos oss? https://steria.sharepoint.com/sites/Personalhndbok/SitePages/1. Hv...",0.030000
9. Ferie.pdf,"Alle ansatte skal avvikle sin hovedferie, minimum fire uker, i juli og ellers i stille perioder. Dette gjelder også medarbeidere som har opparbeidet feriepenger hos tidligere arbeidsgiver. Ferie utover hovedferien skal legges til jul, påske og ellers stille perioder i prosjektet, eller til periode...",0.029710


#### Hybrid Search + Semantic Ranker

Enhance search results using a semantic ranker:

In [6]:
results = search_client.search(
    search_text=searchText,
    vector_queries=[VectorizableTextQuery(text=searchText, k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"],
    query_type="semantic",
    semantic_configuration_name="sopravector2-semantic-configuration"
)

display_results(results)

title,chunk,@search.score,@search.reranker_score
14. Foreldrepermisjon.pdf,"Velferdspermisjon i forbindelse med fødsel Far eller medmor får to uker permisjon med lønn i forbindelse med fødsel. Du informerer din leder, og registrerer dette på 2 weeks in relation to child birth i Workday. Du trenger ikke informere HR om disse to ukene. Merk at permisjonen er to kalenderuker,...",0.030018,2.834420
14. Foreldrepermisjon.pdf,"i permisjon. Goder under foreldrepermisjon Du beholder dine goder som forsikringer, mobilabonnement og pensjon som vanlig mens du er i foreldrepermisjon. Sopra Steria dager tjenes ikke opp mens du er i permisjon. 4/23/25, 9:11 AM 14. Foreldrepermisjon https://steria.sharepoint.com/sites/Persona...",0.025439,2.811734
14. Foreldrepermisjon.pdf,14. Foreldrepermisjon  Hva må du gjøre når du skal ut i…  Endring i foreldrepermisjon  Arbeid i foreldrepermisjon  Syk i foreldrepermisjon  Velferdpermisjon fødsel  Adopsjon  Hva gjelder i permisjonstiden?  Ferie i permisjonstiden  Fri ved amming Hva må du gjøre når du skal ut i foreld...,0.024494,2.703976
14. Foreldrepermisjon.pdf,"du har sendt informasjon til HR, må du gi beskjed om dette i god tid før endringen inntreffer. Dette gjør du ved å korrigere søknaden du har lagt inn i Workday. Du må også søke NAV om endringene. For mors permisjon må du også informere HR dersom barnet blir født før termin – da blir siste dag med fo...",0.022792,2.628828
15. Andre permisjoner.pdf,inn i Workday og sendes til godkjenning hos leder. Nærmeste leder kan etter søknad innvilge inntil tre dager permisjon uten lønn. Ved godkjenning sendes forespørselen til Worktime og genererer din timeføring. Timene går til trekk i lønn inneværende eller påfølgende måned. Lengre permisjoner uten...,0.031319,2.537021


## Troubleshooting

- **Environment Variables Not Loaded:** Ensure you have correctly set the .env file or manually export them in your terminal before running the notebook.
- **Authentication Issues:** If using Managed Identity, make sure your Azure identity has proper role assignments.
- **Search Results Are Empty:** Ensure your Azure AI Search index contains vectorized data.
- **Query Rewriting Issues:** Ensure your search service supports semantic configurations and generative query rewrites.

## Summary

This notebook demonstrates different search techniques using Azure AI Search, including keyword search, vector search, hybrid search, semantic ranking, and query rewriting. The approach enhances search accuracy by leveraging vector embeddings and semantic understanding to retrieve the most relevant documents.

